# Famous CNN Models
This project is aimed at implementing some of the most popular CNN models in keras

### Index
- [LeNet-5](#lenet)
- [AlexNet](#alexnet)
- [VGG-16](#vgg16)

In [1]:
from keras.models import Sequential

from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
from keras.layers import Convolution2D
from keras.layers import AveragePooling2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import ZeroPadding2D

In [3]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
from __future__ import print_function
from PIL import Image
from sklearn.preprocessing import OneHotEncoder

<a id='lenet'></a>
## LeNet-5

In [4]:
from keras.datasets import mnist

In [5]:
train, test = mnist.load_data()

In [50]:
len(train[0])

60000

In [151]:
def process_batch(batch):
    """
    This function takes in a batch of single channel images as array and then resizes
    according to the requirements and returns the processed batch.
    
    Parameters
    ----------
    batch: numpy array 
           Batch of images given for processing as (batch_size, img_width, img_height, channel).
    Returns
    -------
    processed_batch: numpy array
          The processed batch of images.
    
    """
    processed_batch = []
    for img_arr in batch:
        img = Image.fromarray(img_arr)
        img = img.resize((32, 32), Image.ANTIALIAS)
        img_arr = np.reshape(np.array(img), newshape=(32, 32, 1))
        processed_batch.append(img_arr)
        
    return np.array(processed_batch)
        
def create_genrator(data, batch_size):
    """
    This function is used for creating a generator object that iterates over the total input data after processing it.
    The x data is resized.
    The y data is one hot encoded.
    
    Parameters
    ----------
    data              :tuple 
                       Input data in the form of x and y values as a tuple.
    batch_size        :integer
                       The number of batches to which the original data size needs to be split.
    Returns
    -------
    data_generator    :generator object
                       The generator object that supplies in processed x values and the respective y label.
    
    """
    x = data[0]
    y = data[1]
    
    one_h = OneHotEncoder()
    arr = one_h.fit_transform(y.reshape(-1, 1))
    y = np.array(arr.toarray())
    
    index = -1
             
    while(1):
        index = index + batch_size
        processed_batch_x = process_batch(x[index:index+batch_size])
        processed_batch_y = y[index:index+batch_size]
        
        if not(index <= len(x)):
            print(index, sep='\r', end='\r')
            index = -1
        else:
             yield [processed_batch_x, processed_batch_y]       

In [152]:
train_gen = create_genrator(train, 32)

In [153]:
def createLeNet():
    model = Sequential()

    # First convolutional layer with filter = (5*5) and strides = 1
    model.add(Convolution2D(filters=6, kernel_size=(5, 5), input_shape=(32, 32, 1), strides=1, activation='relu'))

    # Average pooling is applied.
    model.add(AveragePooling2D(pool_size=(2, 2), strides=2))

    # Second convolutional layer is applied with the same kernel size and the same strides.
    model.add(Convolution2D(filters=16, kernel_size=(5, 5), strides=1, activation='relu'))

    # A final average pooling layer is applied before using Fully connected layers but with strides=2
    model.add(AveragePooling2D(pool_size=(2, 2), strides=2))

    # We then use fully connected layers

    model.add(Flatten())
    model.add(Dense(units=120))
    model.add(Dense(units=84))
    model.add(Dense(units=10, activation = 'softmax'))

    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    print(model.summary())
    
    return model

In [154]:
LeNet = createLeNet()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 28, 28, 6)         156       
_________________________________________________________________
average_pooling2d_23 (Averag (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 10, 10, 16)        2416      
_________________________________________________________________
average_pooling2d_24 (Averag (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 400)               0         
_________________________________________________________________
dense_34 (Dense)             (None, 120)               48120     
_________________________________________________________________
dense_35 (Dense)             (None, 84)                10164     
__________

In [155]:
history = LeNet.fit_generator(generator=train_gen, 
                              steps_per_epoch=60000/32, 
                              epochs=15)

Epoch 1/15
1875/1875 [==============================] - 9s 5ms/step - loss: 1.9398 - acc: 0.84561
Epoch 2/15
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0762 - acc: 0.9771
Epoch 3/15
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0648 - acc: 0.9805
Epoch 4/15
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0593 - acc: 0.9823
Epoch 5/15
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0560 - acc: 0.98251
Epoch 6/15
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0459 - acc: 0.9862
Epoch 7/15
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0462 - acc: 0.9855
Epoch 8/15
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0401 - acc: 0.9874
Epoch 9/15
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0401 - acc: 0.9882
Epoch 10/15
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0356 - acc: 0.98921
Epoch 11/15
1875/1875 [===

<a id='alexnet'></a>
## AlexNet

In [4]:
input_size = (227, 227)

In [5]:
cifar_generator = ImageDataGenerator(rescale=1./255)

cifar_data = cifar_generator.flow_from_directory('datasets/cifar-10/train', 
                                                 batch_size=32, 
                                                 target_size=input_size, 
                                                 class_mode='categorical')

Found 50000 images belonging to 10 classes.


In [6]:
model = Sequential()

model.add(Convolution2D(filters=96, kernel_size=(11, 11), input_shape=(227, 227, 3), strides=4, activation='relu'))
model.add(MaxPool2D(pool_size=(3 ,3), strides=2))
model.add(BatchNormalization())

model.add(Convolution2D(filters=256, kernel_size=(5, 5), strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(3 ,3), strides=2))
model.add(BatchNormalization())

model.add(Convolution2D(filters=384, kernel_size=(3, 3), strides=1, padding='same', activation='relu'))
model.add(BatchNormalization())

model.add(Convolution2D(filters=384, kernel_size=(3, 3), strides=1, padding='same', activation='relu'))
model.add(BatchNormalization())

model.add(Convolution2D(filters=256, kernel_size=(3, 3), strides=1, padding='same', activation='relu'))
model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=(3 ,3), strides=2))

model.add(Flatten())

model.add(Dense(units=4096, activation='relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(units=4096, activation='relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(units=2, activation='softmax'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 384)       885120    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 384)       1327488   
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 13, 13, 256)       884992    
__________

In [8]:
model.fit_generator(generator=cifar_data, steps_per_epoch=100, epochs=15, workers=4, use_multiprocessing=True)

Epoch 1/15
  3/100 [..............................] - ETA: 6:39 - loss: 9.8345 - acc: 0.1354

Process ForkPoolWorker-1:
Process ForkPoolWorker-3:
Process ForkPoolWorker-4:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/nevin/.conda/envs/tensorflow-ngpu/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/nevin/.conda/envs/tensorflow-ngpu/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/nevin/.conda/envs/tensorflow-ngpu/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/nevin/.conda/envs/tensorflow-ngpu/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/nevin/.conda/envs/tensorflow-ngpu/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/nevin/.conda/envs/tensorflow-ngpu/lib/python3.5/mul

KeyboardInterrupt: 

In [ ]:
from keras.applications.